## 104人力銀行網站爬蟲

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# 設定模擬的 headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.104.com.tw/company/uboh3w8?job=&roleJobCat=0_0&area=0&page=1&pageSize=20&order=8&asc=0&jobsource=checkc&tab=job',
    'Connection': 'keep-alive'
}

# 要爬取的職位列表頁面 URL
url = 'https://www.104.com.tw/jobs/search/api/jobs?indcat=1012001000&isJobList=1&jobcat=2007000000&jobsource=m_joblist_search&page=1&pagesize=20&searchJobs=1' #更改pagesize 可以獲得更多職業



# 發送請求
response = requests.get(url, headers=headers)

time.sleep(3)

# 檢查請求是否成功
if response.status_code == 200:
    # 使用 BeautifulSoup 解析頁面
    data = response.json()
    counter =0
    for job in data["data"]:
        print(job["jobName"])
        print(job["custName"])
        print (counter)
        counter+=1
    print(data)
    

else:
    print(f"Failed to retrieve data from {url}, status code: {response.status_code}")


#### 抓取各式職缺

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [9]:


def get_all_job_info(indcat="1012001000", jobcat="2007000000", pagesize=30):

    # 設定模擬的 headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
        'Referer': 'https://www.104.com.tw',
        'Connection': 'keep-alive'
    }

    # API URL
    api_url = 'https://www.104.com.tw/jobs/search/api/jobs?indcat=' + indcat + '&isJobList=3&jobcat=' + jobcat + '&jobsource=m_joblist_search&page=1&pagesize=' + str(pagesize) + '&searchJobs=1'

    # 發送 API 請求
    response = requests.get(api_url, headers=headers)


    if response.status_code == 200:
        data = response.json()
        print(data)
        
        job_list = []
        counter = 0
        for job in data["data"]:
            job_name = job["jobName"]
            cust_name = job["custName"]
            job_link = job["link"]["job"]

            print(f"{counter}. Job Name: {job_name}, Company: {cust_name}, URL: {job_link}")
            job_list.append({"Job Name": job_name, "Company": cust_name, "URL": job_link})
            
            
            # 嘗試抓取薪資資訊
            try:
                job_response = requests.get(job_link, headers=headers)
                if job_response.status_code == 200:
                    job_soup = BeautifulSoup(job_response.text, 'lxml')

                    # 使用正則表達式搜尋薪資資訊
                    salary_tag = job_soup.find(text=re.compile(r'月薪\s*\d+,\d+~\d+,\d+\s*元')) 
                    if salary_tag:
                        salary_info = salary_tag.strip()  # 去除多餘空白
                        print(f"   Salary: {salary_info}")
                    else:
                        # 若找不到明確的薪資範圍，則顯示其他資訊
                        salary_tag = job_soup.find('div', class_='job-description__salary')
                        if salary_tag:
                            salary_info = salary_tag.get_text(strip=True)
                            print(f"   Salary: {salary_info}")
                        else:
                            print("   Salary: Not found")
            except Exception as e:  # 加入 except 區塊處理異常
                print(f"   Error fetching salary info: {e}")

            counter += 1
            # 避免被封鎖，延遲請求
            time.sleep(1)
        jobs_df = pd.DataFrame(job_list)
        return jobs_df

    else:
        print(f"Failed to retrieve data from {api_url}, status code: {response.status_code}")


def get_company_job_list(company_name, jobCat):

    # 設定模擬的 headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
        'Referer': 'https://www.104.com.tw',
        'Connection': 'keep-alive'
    }

    # API URL
    company_url = 'https://www.104.com.tw/company/ajax/list?jobsource=tab_job_to_cs&keyword=' + company_name + '&page=1&pageSize=12'

    # 發送 API 請求
    response_company = requests.get(company_url, headers=headers)
    if response_company.status_code == 200:
        data = response_company.json()  
    else:
        print(f"Failed to retrieve data from {company_url}, status code: {response_company.status_code}")
        return False
    
    if data.get('data', []) == []:
        print(f"Company {company_name} not found.")
        return False
    else:
        companyCode = data.get('data', [])[0].get('encodedCustNo')
        companyJobsurl = 'https://www.104.com.tw/company/ajax/joblist/' + companyCode + '?job=' + jobCat
        response_companyJobs = requests.get(companyJobsurl, headers=headers)
        
    if response_companyJobs.status_code == 200:
        companyJobs = response_companyJobs.json()  

    else:
        print(f"Failed to retrieve data from {company_url}, status code: {response_companyJobs.status_code}")
        return False
    
    job_info_list = []
    norm_jobs = companyJobs.get('data', {}).get('list', {}).get('normalJobs', [])
    top_jobs = companyJobs.get('data', {}).get('list', {}).get('topJobs', [])
    for job in norm_jobs:
        job_info = {
            'Company': company_name,
            'Job Name': job.get('jobName'),
            'URL': 'https://' + job.get('jobUrl', '').lstrip('/')
        }
        job_info_list.append(job_info)
        
    for job in top_jobs:
        job_info = {
            'Company': company_name,
            'Job Name': job.get('jobName'),
            'URL': 'https://' + job.get('jobUrl', '').lstrip('/')
        }
        job_info_list.append(job_info)
    # jobs_df = pd.DataFrame(job_info_list)
    return job_info_list


# 定義 URL 和 Headers
def get_job_detail(job_code):
    url = "https://www.104.com.tw/job/ajax/content/" + job_code
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Referer": "https://www.104.com.tw/job/" + job_code
    }
    print(url)
    try:
        # 發送 GET 請求
        response = requests.get(url, headers=headers)
        
        # 檢查狀態碼
        if response.status_code == 200:
            # 將返回的 JSON 資料解析為 Python 字典
            data = response.json()
            print(data)
            # 提取需要的字段
            job_name = data.get('data', {}).get('header', {}).get('jobName', 'Not found')
            company_name = data.get('data', {}).get('header', {}).get('custName', 'Not found')
            salary = data.get('data', {}).get('jobDetail', {}).get('salary', 'Not found')
            job_description = data.get('data', {}).get('jobDetail', {}).get('jobDescription', 'Not found')
            job_welfare = data.get('data', {}).get('welfare', {}).get('welfare', 'Not found')
            # legal_tags = data.get('data', {}).get('legalTag', [])
            # other_welfare_tags = data.get('data', {}).get('welfare', {}).get('tag', [])
            # 整理輸出格式
            # print("Job Information")
            # print("====================")
            # print(f"Job Name       : {job_name}")
            # print(f"Company Name   : {company_name}")
            # print(f"Salary         : {salary}")
            # print(f"Description    :\n{job_description}")
            # print(f"Welfare    :\n{job_welfare}")
            return {
                "Job Name": job_name,
                "Company Name": company_name,
                "Salary": salary,
                "Description": job_description,
                "Welfare": job_welfare
                # "Legal_tags": legal_tags,
                # "other_Welfare_tags": other_welfare_tags
            }
            
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")


# get_job_detail("6cbqb")

## 工程師

#### 抓取所有醫院的職缺

In [ ]:
engineer_jobs_df =  get_all_job_info(indcat="1012001001", jobcat="2007000000", pagesize=50)
engineer_jobs_df

#### 抓取特定醫院職缺

In [11]:

# Read Excel with correct parameters
hospital_csv = pd.read_excel(
    "104人力銀行職務登錄表市場比較醫療院所 (1).xlsx",
    skiprows=1,  # Skip the first row with Chinese headers
    names=['序號', '機構代碼', '機構名稱', '地址'],  # Assign proper column names
    usecols=[0, 1, 2, 3]  # Select columns by index
)

all_jobs = []  # Initialize empty list
for index, row in hospital_csv.iterrows():
    if pd.notna(row['機構名稱']):  # Check if name is not NaN
        print(f"{index}. {row['機構名稱']}")

    hospital_jobs_list = get_company_job_list(row['機構名稱'], jobCat="資訊")
    if hospital_jobs_list:
        print(hospital_jobs_list)
        all_jobs.extend(hospital_jobs_list)
    time.sleep(2)
    
engineer_jobs_df = pd.DataFrame(all_jobs)
engineer_jobs_df

0. 天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院


[{'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '程式設計師(資訊部)', 'URL': 'https://www.104.com.tw/job/26906'}, {'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '商業智慧(BI) 數據分析師(資訊部)', 'URL': 'https://www.104.com.tw/job/8hxuh'}]
1. 戴德森醫療財團法人嘉義基督教醫院
[{'Company': '戴德森醫療財團法人嘉義基督教醫院', 'Job Name': '資訊室資訊管理師', 'URL': 'https://www.104.com.tw/job/50zry'}]
2. 佛教慈濟醫療財團法人大林慈濟醫院
[{'Company': '佛教慈濟醫療財團法人大林慈濟醫院', 'Job Name': '大林慈院-智慧醫療創新中心-軟體程式設計師（資訊系統）', 'URL': 'https://www.104.com.tw/job/7doch'}, {'Company': '佛教慈濟醫療財團法人大林慈濟醫院', 'Job Name': '醫療法人資訊室軟體開發工程師（工作地點：大林慈濟醫院）', 'URL': 'https://www.104.com.tw/job/7xe54'}, {'Company': '佛教慈濟醫療財團法人大林慈濟醫院', 'Job Name': '醫療法人資訊室系統工程師（工作地點：大林慈濟醫院）', 'URL': 'https://www.104.com.tw/job/8lcx8'}]
3. 衛生福利部嘉義醫院
4. 臺中榮民總醫院嘉義分院
5. 陽明醫院
6. 長庚醫療財團法人嘉義長庚紀念醫院
7. 台灣基督長老教會新樓醫療財團法人麻豆新樓醫院
Company 台灣基督長老教會新樓醫療財團法人麻豆新樓醫院 not found.
8. 奇美醫療財團法人柳營奇美醫院
9. 國立成功大學醫學院附設醫院
10. 奇美醫療財團法人奇美醫院
11. 臺南市立安南醫院-委託中國醫藥大學興建經營
12. 郭綜合醫院
[{'Company': '郭綜合醫院', 'Job N

,Company,Job Name,URL
0,天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院,程式設計師(資訊部),https://www.104.com.tw/job/26906
1,天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院,商業智慧(BI) 數據分析師(資訊部),https://www.104.com.tw/job/8hxuh
2,戴德森醫療財團法人嘉義基督教醫院,資訊室資訊管理師,https://www.104.com.tw/job/50zry
3,佛教慈濟醫療財團法人大林慈濟醫院,大林慈院-智慧醫療創新中心-軟體程式設計師（資訊系統）,https://www.104.com.tw/job/7doch
4,佛教慈濟醫療財團法人大林慈濟醫院,醫療法人資訊室軟體開發工程師（工作地點：大林慈濟醫院）,https://www.104.com.tw/job/7xe54
5,佛教慈濟醫療財團法人大林慈濟醫院,醫療法人資訊室系統工程師（工作地點：大林慈濟醫院）,https://www.104.com.tw/job/8lcx8
6,郭綜合醫院,資訊室_系統工程師,https://www.104.com.tw/job/7rppj
7,天主教若瑟醫療財團法人若瑟醫院,資訊護理師,https://www.104.com.tw/job/8j9l3
8,秀傳醫療社團法人秀傳紀念醫院,資訊維護工程師(路竹),https://www.104.com.tw/job/87ltw
9,秀傳醫療社團法人秀傳紀念醫院,資訊維護工程師(彰濱),https://www.104.com.tw/job/834n3


In [12]:
import time
job_details_list = []
for url in engineer_jobs_df['URL']:
    job_code = url.split('/')[-1]
    print(job_code)
    job_details = get_job_detail(job_code)
    job_details_list.append(job_details)
    time.sleep(2)
    
job_details_df = pd.DataFrame(job_details_list)
job_details_df.to_csv('engineer_job_details.csv', index=False, encoding='utf-8-sig')

26906
https://www.104.com.tw/job/ajax/content/26906
{'data': {'corpImageRight': {'corpImageRight': {'imageUrl': '', 'link': ''}}, 'header': {'corpImageTop': {'imageUrl': '', 'link': ''}, 'jobName': '程式設計師(資訊部)', 'appearDate': '2025/01/20', 'custName': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'custUrl': 'https://www.104.com.tw/company/uboh3w8', 'analysisType': 1, 'analysisUrl': '//www.104.com.tw/jobs/apply/analysis/26906', 'isSaved': False, 'isApplied': False, 'applyDate': '', 'userApplyCount': 0, 'isActivelyHiring': False}, 'contact': {'hrName': '彭盈森 先生', 'email': '', 'visit': '', 'phone': [], 'other': '應徵本職缺請至「天主教聖馬爾定醫院網站→關於本院→人才招募→人才招募履歷」登錄履歷https://flw.stm.org.tw/AppJob/Login.aspx \n參加面試前，請攜帶書面之登錄『履歷表』及『自傳』。', 'reply': ''}, 'environmentPic': {'environmentPic': [{'thumbnailLink': 'https://static.104.com.tw/b_profile/cust_picture/7000/66009707000/env/s_890328564411235159.jpg?v=20200730174232', 'link': 'https://static.104.com.tw/b_profile/cust_picture/7000/66009707000/env/l_890328564411235159.jpg?

#### 護理師

In [ ]:
nurse_jobs_df =  get_all_job_info( indcat="1012001001", jobcat="2015001004",pagesize=50)
nurse_jobs_df

#### 抓取特定醫院職缺

In [13]:
# Read Excel with correct parameters
hospital_csv = pd.read_excel(
    "104人力銀行職務登錄表市場比較醫療院所 (1).xlsx",
    skiprows=1,  # Skip the first row with Chinese headers
    names=['序號', '機構代碼', '機構名稱', '地址'],  # Assign proper column names
    usecols=[0, 1, 2, 3]  # Select columns by index
)

all_jobs = []  # Initialize empty list
for index, row in hospital_csv.iterrows():
    if pd.notna(row['機構名稱']):  # Check if name is not NaN
        print(f"{index}. {row['機構名稱']}")

    hospital_jobs_list = get_company_job_list(row['機構名稱'], jobCat="護理師")
    if hospital_jobs_list:
        print(hospital_jobs_list)
        all_jobs.extend(hospital_jobs_list)
    time.sleep(2)
    
nurse_jobs_df = pd.DataFrame(all_jobs)
nurse_jobs_df

0. 天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院
[{'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '護理師(急診)', 'URL': 'https://www.104.com.tw/job/7oamh'}, {'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '護理師(產房)', 'URL': 'https://www.104.com.tw/job/7pz9p'}, {'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '護理師(麻醉部)', 'URL': 'https://www.104.com.tw/job/89k9e'}, {'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '護理師(嬰兒室)', 'URL': 'https://www.104.com.tw/job/7xmua'}, {'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '護理師(注射室)', 'URL': 'https://www.104.com.tw/job/3k2fb'}, {'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '護理師(開刀房)', 'URL': 'https://www.104.com.tw/job/hor1'}, {'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '護理師(內科病房)', 'URL': 'https://www.104.com.tw/job/6uacp'}, {'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '護理師(外科病房)', 'URL': 'https://www.104.com.tw/job/5p38t'}, {'Company': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'Job Name': '護理師(兒科病房)', 'URL': 'https://www.104.com.tw/jo

,Company,Job Name,URL
0,天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院,護理師(急診),https://www.104.com.tw/job/7oamh
1,天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院,護理師(產房),https://www.104.com.tw/job/7pz9p
2,天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院,護理師(麻醉部),https://www.104.com.tw/job/89k9e
3,天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院,護理師(嬰兒室),https://www.104.com.tw/job/7xmua
4,天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院,護理師(注射室),https://www.104.com.tw/job/3k2fb
...,...,...,...
221,義大醫療財團法人義大醫院,兒科病房護理師,https://www.104.com.tw/job/87xlr
222,義大醫療財團法人義大醫院,婦產科病房護理師,https://www.104.com.tw/job/87xlz
223,義大醫療財團法人義大醫院,病房護理師(義大大昌醫院),https://www.104.com.tw/job/6lnfc
224,義大醫療財團法人義大醫院,病房護理師,https://www.104.com.tw/job/7xo3d


In [14]:
import time
job_details_list = []
for url in nurse_jobs_df['URL']:
    job_code = url.split('/')[-1]
    print(job_code)
    job_details = get_job_detail(job_code)
    job_details_list.append(job_details)
    time.sleep(3)
    
job_details_df = pd.DataFrame(job_details_list)
job_details_df.to_csv('nurse_jobs_details.csv', index=False, encoding='utf-8-sig')

7oamh
https://www.104.com.tw/job/ajax/content/7oamh
{'data': {'corpImageRight': {'corpImageRight': {'imageUrl': '', 'link': ''}}, 'header': {'corpImageTop': {'imageUrl': '', 'link': ''}, 'jobName': '護理師(急診)', 'appearDate': '2025/01/15', 'custName': '天主教中華聖母修女會醫療財團法人天主教聖馬爾定醫院', 'custUrl': 'https://www.104.com.tw/company/uboh3w8', 'analysisType': 1, 'analysisUrl': '//www.104.com.tw/jobs/apply/analysis/7oamh', 'isSaved': False, 'isApplied': False, 'applyDate': '', 'userApplyCount': 0, 'isActivelyHiring': False}, 'contact': {'hrName': '彭盈森 先生', 'email': '', 'visit': '', 'phone': [], 'other': '應徵本職缺請至「天主教聖馬爾定醫院網站→關於本院→人才招募→人才招募履歷」登錄履歷https://flw.stm.org.tw/AppJob/Login.aspx\n參加面試前，請攜帶書面之登錄『履歷表』及『自傳』。', 'reply': ''}, 'environmentPic': {'environmentPic': [{'thumbnailLink': 'https://static.104.com.tw/b_profile/cust_picture/7000/66009707000/env/s_890328564411235159.jpg?v=20200730174232', 'link': 'https://static.104.com.tw/b_profile/cust_picture/7000/66009707000/env/l_890328564411235159.jpg?v=20